<a href="https://colab.research.google.com/github/YuricgSilva/Pre-processamento_Realce_de_brilho_e_contraste/blob/main/Pr%C3%A9_processamento_Realce_de_brilho_e_contraste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#Import`s
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from google.colab.patches import cv2_imshow

In [114]:
img_path = "/content/images/mona_lisa.pgm"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(5, 5))
axs.imshow(img, cmap='gray')
axs.axis('off')
plt.show()

REALCE DE BRILHO E CONTRASTE UTILIZANDO A TRANSFORMAÇÃO LINEAR DA RETA.

In [ ]:
# Função para aplicar transformação linear
def linear_transformation(img, a, b):
  altura, largura = img.shape
  for i in range(altura):
      for j in range(largura):
          novo_valor = (a*int(img[i, j])+b)
          img[i, j] = max(0, min(255, novo_valor))
  return img.astype(np.uint8)

# Parâmetros (a, b)
params = [
    (1, 0), (1, 50), (1, 100),
    (0.15, 0), (0.15, 50), (0.15, 100),
    (2, 0), (2, 50), (2, 100)
]

# Aplicar transformações e armazenar resultados
resultados = []
for i, (a, b) in enumerate(params):
    original = np.array(img)
    img_transf = linear_transformation(original, a, b)
    hist, _ = np.histogram(img_transf.flatten(), bins=256, range=[0, 256])
    resultados.append((a, b, img_transf, hist))

# Plotar imagens e histogramas
fig, axes = plt.subplots(len(resultados), 2, figsize=(10, 30))
fig.suptitle("Transformações Lineares: Imagens e Histogramas", fontsize=16)

for idx, (a, b, img, hist) in enumerate(resultados):
    ax_img, ax_hist = axes[idx]
    ax_img.imshow(img, cmap='gray')
    ax_img.set_title(f"a = {a}, b = {b}")
    ax_img.axis("off")

    ax_hist.plot(hist, color='black')
    ax_hist.set_title(f"Histograma - a = {a}, b = {b}")
    ax_hist.set_xlim([0, 255])

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

TRANSFORMAÇÃO DE AUTOESCALA

In [ ]:
def autoscale_transformation(img):
    img_float = img.astype(np.float32) / 255.0
    Lmin = np.min(img_float)
    Lmax = np.max(img_float)
    if Lmax - Lmin == 0:
        return np.zeros_like(img, dtype=np.uint8)
    img_scaled = (img_float - Lmin) * (255 / (Lmax - Lmin))
    return img_scaled.astype(np.uint8)

resultados_autoescala = []
for a, b, img, hist in resultados:
    img_auto = autoscale_transformation(img)
    hist_auto, _ = np.histogram(img_auto.flatten(), bins=256, range=[0, 256])
    resultados_autoescala.append((a, b, img_auto, hist_auto))

fig, axes = plt.subplots(len(resultados_autoescala), 2, figsize=(10, 30))
fig.suptitle("Transformações Lineares com Autoescala: Imagens e Histogramas", fontsize=16)

for idx, (a, b, img, hist) in enumerate(resultados_autoescala):
    ax_img, ax_hist = axes[idx]
    ax_img.imshow(img, cmap='gray')
    ax_img.set_title(f"a = {a}, b = {b} (Autoescala)")
    ax_img.axis("off")

    ax_hist.plot(hist, color='black')
    ax_hist.set_title(f"Histograma - a = {a}, b = {b} (Autoescala)")
    ax_hist.set_xlim([0, 255])

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


TRANSFORMAÇÃO NEGATIVA

In [ ]:
def negative_transformation(img):
    return 255 - img

img_negativa = negative_transformation(img)

hist_negativa, _ = np.histogram(img_negativa.flatten(), bins=256, range=[0, 256])

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(img_negativa, cmap='gray')
axes[0].set_title("Imagem Negativa")
axes[0].axis("off")

axes[1].plot(hist_negativa, color='black')
axes[1].set_title("Histograma da Imagem Negativa")
axes[1].set_xlim([0, 255])

plt.tight_layout()
plt.show()


TRANSFORMAÇÃO LOGARÍTMICA

In [ ]:
def intensidade_logaritmica(img, constante):
    img_float = img.astype(np.float32)
    img_log = constante * np.log10(1 + img_float)
    img_log = np.clip(img_log, 0, 255)
    return img_log.astype(np.uint8)

constants = [1, 25, 50, 100]
for c in constants:
    img_log = intensidade_logaritmica(img, c)
    hist_log, _ = np.histogram(img_log.flatten(), bins=256, range=[0, 256])

    fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 6)) # Create a subplot with 1 row and 2 columns

    ax_img.imshow(img_log, cmap='gray') # Display the image on the left subplot
    ax_img.set_title(f"Imagem com Transformação Logarítmica (c = {c})")
    ax_img.axis("off")

    ax_hist.plot(hist_log, color='black') # Display the histogram on the right subplot
    ax_hist.set_title(f"Histograma (c = {c})")
    ax_hist.set_xlabel("Níveis de Intensidade")
    ax_hist.set_ylabel("Frequência")
    ax_hist.set_xlim([0, 255])

    plt.show()

TRANSFORMAÇÃO EXPONENCIAL

In [ ]:
def transformacao_exponencial(img, constante, gama):
    img_float = img.astype(np.float32)
    img_exp = constante * np.power(img_float, gama)
    img_exp = np.clip(img_exp, 0, 255)
    return img_exp.astype(np.uint8)

parameters = [
    (1, 0.15), (1, 1), (1, 2),
    (30, 0.15), (30, 1), (30, 2)
]

for c, gamma in parameters:
    img_exp = transformacao_exponencial(img, c, gamma)
    hist_exp, _ = np.histogram(img_exp.flatten(), bins=256, range=[0, 256])

    fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 6))

    ax_img.imshow(img_exp, cmap='gray')
    ax_img.set_title(f"Imagem com Transformação Exponencial (c = {c}, γ = {gamma})")
    ax_img.axis("off")

    ax_hist.plot(hist_exp, color='black')
    ax_hist.set_title(f"Histograma (c = {c}, γ = {gamma})")
    ax_hist.set_xlabel("Níveis de Intensidade")
    ax_hist.set_ylabel("Frequência")
    ax_hist.set_xlim([0, 255])

    plt.show()


TRANSFORMAÇÃO QUADRADO

In [ ]:
def transformacao_quadrada(img, constante):
    img_float = img.astype(np.float32)
    img_quad = constante * np.power(img_float, 2)
    img_quad = np.clip(img_quad, 0, 255)
    return img_quad.astype(np.uint8)

constants = [1, 25, 50, 100]
for c in constants:
    img_quad = transformacao_quadrada(img, c)
    hist_quad, _ = np.histogram(img_quad.flatten(), bins=256, range=[0, 256])

    fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 6))

    ax_img.imshow(img_quad, cmap='gray')
    ax_img.set_title(f"Imagem com Transformação Quadrada (c = {c})")
    ax_img.axis("off")

    ax_hist.plot(hist_quad, color='black')
    ax_hist.set_title(f"Histograma (c = {c})")
    ax_hist.set_xlabel("Níveis de Intensidade")
    ax_hist.set_ylabel("Frequência")
    ax_hist.set_xlim([0, 255])

    plt.show()


TRANSFORMAÇÃO RAIZ QUADRADA

In [ ]:
def transformacao_raiz_quadrada(img, constante):
    img_float = img.astype(np.float32)
    img_sqrt = constante * np.sqrt(img_float)
    img_sqrt = np.clip(img_sqrt, 0, 255)

    return img_sqrt.astype(np.uint8)
constants = [1, 25, 50, 100]
for c in constants:
    img_sqrt = transformacao_raiz_quadrada(img, c)
    hist_sqrt, _ = np.histogram(img_sqrt.flatten(), bins=256, range=[0, 256])

    fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 6))

    ax_img.imshow(img_sqrt, cmap='gray')
    ax_img.set_title(f"Imagem com Transformação Raiz Quadrada (c = {c})")
    ax_img.axis("off")

    ax_hist.plot(hist_sqrt, color='black')
    ax_hist.set_title(f"Histograma (c = {c})")
    ax_hist.set_xlabel("Níveis de Intensidade")
    ax_hist.set_ylabel("Frequência")
    ax_hist.set_xlim([0, 255])

    plt.show()

EQUALIZAÇÃO DE HISTOGRAMA

In [ ]:
def equalizacao_de_histograma(img):
    histograma = np.bincount(img.flatten(), minlength=256).astype(np.float32)
    total_pixels = img.size
    pdf = histograma / total_pixels
    cdf = np.cumsum(pdf)
    cdf_normalizada = np.round(cdf * 255).astype(np.uint8)
    imagem_equalizada = cdf_normalizada[img]

    return imagem_equalizada

img_equalizada = equalizacao_de_histograma(img)

hist_equalizado, _ = np.histogram(img_equalizada.flatten(), bins=256, range=[0, 256])

fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 6))

ax_img.imshow(img_equalizada, cmap='gray')
ax_img.set_title(f"Imagem Equalizada")
ax_img.axis("off")

ax_hist.plot(hist_equalizado, color='black')
ax_hist.set_title(f"Histograma Equalizado")
ax_hist.set_xlabel("Níveis de Intensidade")
ax_hist.set_ylabel("Frequência")
ax_hist.set_xlim([0, 255])

plt.show()
